<a href="https://colab.research.google.com/github/Arta-DS/DS/blob/main/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание 1.**

Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
Сделать нейронную сеть
Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
Проверить качество

**Задание 2.**

Выполнить практическую работу из лекционного ноутбука.

Построить RNN-ячейку на основе полносвязных слоев
Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”


In [17]:
import numpy as np
import string
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical

import pandas as pd
import time
import torch
import torch.nn as nn

In [18]:
ALPHABET = string.ascii_lowercase
CHAR_TO_IDX = {ch: i for i, ch in enumerate(ALPHABET)}
IDX_TO_CHAR = {i: ch for i, ch in enumerate(ALPHABET)}
VOCAB_SIZE = len(ALPHABET)
MAX_LEN = 10
NUM_SHIFTS = 25  # сдвиги от 1 до 25

In [19]:
def caesar_cipher(text, shift):
    return ''.join(
        IDX_TO_CHAR[(CHAR_TO_IDX[ch] + shift) % VOCAB_SIZE] if ch in CHAR_TO_IDX else ch
        for ch in text.lower()
    )

def caesar_decipher(text, shift):
    return ''.join(
        IDX_TO_CHAR[(CHAR_TO_IDX[ch] - shift) % VOCAB_SIZE] if ch in CHAR_TO_IDX else ch
        for ch in text.lower()
    )

In [20]:
WORD_LIST = [
    "hello", "world", "apple", "zebra", "python", "cipher", "secret", "model", "train", "learn",
    "brain", "neural", "network", "letter", "shift", "decode", "encode", "attack", "system", "input",
    "output", "vector", "matrix", "tensor", "layer", "weight", "bias", "epoch", "batch", "data",
    "text", "word", "code", "logic", "solve", "answer", "right", "wrong", "quick", "brown",
    "jumps", "over", "lazy", "dog", "house", "green", "black", "white", "color", "music",
    "water", "earth", "flame", "light", "sound", "voice", "image", "pixel", "frame", "video"
]
WORD_LIST = [w.lower() for w in WORD_LIST if w.isalpha() and 3 <= len(w) <= MAX_LEN]

In [21]:
def generate_shift_dataset(n_samples=30000):
    X_texts = []
    y_shifts = []
    for _ in range(n_samples):
        plain = np.random.choice(WORD_LIST)
        shift = np.random.randint(1, 26)  # 1..25
        cipher = caesar_cipher(plain, shift)
        X_texts.append(cipher.ljust(MAX_LEN)[:MAX_LEN])
        y_shifts.append(shift - 1)  # классы от 0 до 24
    return X_texts, np.array(y_shifts)

In [22]:
def text_to_tensor(texts):
    tensor = np.zeros((len(texts), MAX_LEN, VOCAB_SIZE), dtype=np.float32)
    for i, text in enumerate(texts):
        for j, ch in enumerate(text):
            if ch in CHAR_TO_IDX:
                tensor[i, j, CHAR_TO_IDX[ch]] = 1.0
    return tensor

In [23]:
inputs = Input(shape=(MAX_LEN, VOCAB_SIZE))
x = LSTM(128, return_sequences=True)(inputs)
x = GlobalMaxPooling1D()(x)  # сводим к вектору
x = Dense(64, activation='relu')(x)
outputs = Dense(NUM_SHIFTS, activation='softmax')(x)  # 25 классов

shift_model = Model(inputs, outputs)
shift_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
shift_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 10, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 10, 128)        │        79,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25)             │         1,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,241 (348.60 KB)

 Trainable params: 89,241 (348.60 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
print("Генерация данных...")
X_texts, y_shifts = generate_shift_dataset(30000)
X = text_to_tensor(X_texts)

print("Обучение модели предсказания сдвига...")
shift_model.fit(X, y_shifts, batch_size=64, epochs=30, validation_split=0.1)

# === Дешифровка с использованием предсказанного сдвига ===
def decrypt_with_shift_model(cipher_text):
    padded = cipher_text.ljust(MAX_LEN)[:MAX_LEN]
    inp = text_to_tensor([padded])
    pred_shift_idx = np.argmax(shift_model.predict(inp, verbose=0), axis=1)[0]
    pred_shift = pred_shift_idx + 1  # обратно к 1..25
    return caesar_decipher(cipher_text, pred_shift)

Генерация данных...
Обучение модели предсказания сдвига...
Epoch 1/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.1701 - loss: 2.8075 - val_accuracy: 0.4960 - val_loss: 1.6235
Epoch 2/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.5790 - loss: 1.3710 - val_accuracy: 0.7623 - val_loss: 0.8066
Epoch 3/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.8389 - loss: 0.6256 - val_accuracy: 0.9360 - val_loss: 0.3044
Epoch 4/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9722 - loss: 0.2076 - val_accuracy: 1.0000 - val_loss: 0.0749
Epoch 5/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9994 - loss: 0.0580 - val_accuracy: 1.0000 - val_loss: 0.0289
Epoch 6/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 1.0000 - loss: 0.0228 - val_accuracy: 1.0000 - val_loss: 0.0144
Epoch 7/30
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 1.0000 - val_loss: 0.0084
Epoch 8/30
422/422 ━━━━━━━━━━━━━━━━━━

In [25]:
# === Тестирование ===
test_cases = [
    ("hello", 3),
    ("world", 7),
    ("python", 13),
    ("secret", 25),
    ("neural", 10)
]

print("\n=== Тестирование ===")
for plain, shift in test_cases:
    cipher = caesar_cipher(plain, shift)
    decrypted = decrypt_with_shift_model(cipher)
    success = decrypted == plain
    print(f"Оригинал: {plain:8} | Сдвиг: {shift:2} | Шифр: {cipher:10} | Расшифровка: {decrypted:8} | Успех: {success}")


=== Тестирование ===
Оригинал: hello    | Сдвиг:  3 | Шифр: khoor      | Расшифровка: hello    | Успех: True
Оригинал: world    | Сдвиг:  7 | Шифр: dvysk      | Расшифровка: world    | Успех: True
Оригинал: python   | Сдвиг: 13 | Шифр: clguba     | Расшифровка: python   | Успех: True
Оригинал: secret   | Сдвиг: 25 | Шифр: rdbqds     | Расшифровка: secret   | Успех: True
Оригинал: neural   | Сдвиг: 10 | Шифр: xoebkv     | Расшифровка: neural   | Успех: True


==============================================================================

In [28]:
# 1. Загрузка и подготовка данных
df = pd.read_csv('simpsons_script_lines.csv')
phrases = df['normalized_text'].dropna().tolist()

# Оставим только строчные буквы и пробел
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['<PAD>', '<UNK>'] + sorted(CHARS)  # 0: PAD, 1: UNK, 2+: символы
CHAR_TO_INDEX = {ch: i for i, ch in enumerate(INDEX_TO_CHAR)}
VOCAB_SIZE = len(INDEX_TO_CHAR)

print(f"Размер словаря: {VOCAB_SIZE}")  # должно быть 28 (26 букв + пробел + 2 спецсимвола)

# Преобразуем фразы в последовательности индексов
MAX_LEN = 50
sequences = []
for ph in phrases:
    if not isinstance(ph, str):
        continue
    # Оставляем только разрешённые символы
    clean_ph = ''.join(ch if ch in CHARS else ' ' for ch in ph.lower())
    if len(clean_ph) < 2:
        continue
    seq = [CHAR_TO_INDEX.get(ch, CHAR_TO_INDEX['<UNK>']) for ch in clean_ph]
    sequences.append(seq)

# Создаём тензор X с паддингом
X = torch.full((len(sequences), MAX_LEN), CHAR_TO_INDEX['<PAD>'], dtype=torch.long)
for i, seq in enumerate(sequences):
    truncated = seq[:MAX_LEN]
    X[i, :len(truncated)] = torch.tensor(truncated, dtype=torch.long)

print(f"Форма данных: {X.shape}")

/tmp/ipython-input-3031060131.py:2: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('simpsons_script_lines.csv')


Размер словаря: 29
Форма данных: torch.Size([132075, 50])


In [29]:
#  2. Ручная RNN-ячейка на основе полносвязных слоёв
class SimpleRNNCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        # W_hh * h_{t-1} + W_xh * x_t + b
        self.weight_hh = nn.Linear(hidden_size, hidden_size, bias=True)
        self.weight_xh = nn.Linear(input_size, hidden_size, bias=False)  # bias уже в weight_hh

    def forward(self, x, hidden):
        # x: (batch, input_size)
        # hidden: (batch, hidden_size)
        h_new = torch.tanh(self.weight_hh(hidden) + self.weight_xh(x))
        return h_new

In [30]:
#  3. Многослойная RNN на основе нашей ячейки
class ManualRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.layers = nn.ModuleList([
            SimpleRNNCell(input_size if i == 0 else hidden_size, hidden_size)
            for i in range(num_layers)
        ])

    def forward(self, x, hidden=None):
        # x: (batch, seq_len, input_size)
        batch, seq_len, _ = x.shape
        if hidden is None:
            hidden = [torch.zeros(batch, self.hidden_size, device=x.device) for _ in range(self.num_layers)]
        outputs = []
        for t in range(seq_len):
            inp = x[:, t, :]  # (batch, input_size)
            for layer_idx, layer in enumerate(self.layers):
                inp = layer(inp, hidden[layer_idx])
                hidden[layer_idx] = inp
            outputs.append(inp)
        outputs = torch.stack(outputs, dim=1)  # (batch, seq_len, hidden_size)
        return outputs, hidden

In [31]:
#  4. Полная модель для генерации текста
class SimpsonsRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=30, hidden_size=128, num_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=CHAR_TO_INDEX['<PAD>'])
        self.rnn = ManualRNN(embed_dim, hidden_size, num_layers)
        self.fc_out = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        # x: (batch, seq_len)
        embedded = self.embedding(x)  # (batch, seq_len, embed_dim)
        rnn_out, hidden = self.rnn(embedded, hidden)
        logits = self.fc_out(rnn_out)  # (batch, seq_len, vocab_size)
        return logits, hidden

In [27]:
#  5. Обучение
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpsonsRNN(VOCAB_SIZE, embed_dim=30, hidden_size=128, num_layers=1).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=CHAR_TO_INDEX['<PAD>'])
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Обучение
EPOCHS = 20
BATCH_SIZE = 128

for ep in range(EPOCHS):
    start = time.time()
    model.train()
    total_loss = 0.0
    num_batches = 0

    # Перемешиваем индексы
    indices = torch.randperm(X.size(0))

    for i in range(0, len(indices), BATCH_SIZE):
        batch_indices = indices[i:i+BATCH_SIZE]
        batch = X[batch_indices].to(device)

        # X_batch: все символы, кроме последнего
        # Y_batch: все символы, кроме первого
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].contiguous()

        optimizer.zero_grad()
        logits, _ = model(X_batch)
        # logits: (batch, seq_len, vocab_size) -> (batch*seq_len, vocab_size)
        # Y_batch: (batch, seq_len) -> (batch*seq_len,)
        loss = criterion(logits.view(-1, VOCAB_SIZE), Y_batch.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch {ep+1}/{EPOCHS}. Time: {time.time() - start:.2f}s, Loss: {avg_loss:.4f}")

# === 6. Генерация текста ===
def generate_text(seed_text, max_length=50, temperature=1.0):
    model.eval()
    with torch.no_grad():
        # Преобразуем seed в индексы
        current_seq = [CHAR_TO_INDEX.get(ch, CHAR_TO_INDEX['<UNK>']) for ch in seed_text.lower() if ch in CHARS or ch == ' ']
        if not current_seq:
            current_seq = [CHAR_TO_INDEX['h']]  # fallback

        input_tensor = torch.tensor([current_seq], dtype=torch.long).to(device)
        hidden = None

        # Прогоняем seed через модель, чтобы инициализировать hidden state
        _, hidden = model(input_tensor, hidden)
        last_char = input_tensor[0, -1].item()

        generated = list(seed_text)

        for _ in range(max_length):
            # Подаём последний символ
            input_char = torch.tensor([[last_char]], device=device)
            logits, hidden = model(input_char, hidden)

            # Применяем температуру
            logits = logits.squeeze() / temperature
            probs = torch.softmax(logits, dim=-1)

            # Сэмплируем следующий символ
            next_char_idx = torch.multinomial(probs, 1).item()

            if next_char_idx == CHAR_TO_INDEX['<PAD>']:
                break

            next_char = INDEX_TO_CHAR[next_char_idx]
            if next_char == '<UNK>':
                continue

            generated.append(next_char)
            last_char = next_char_idx

            # Остановка при генерации "мусора"
            if len(generated) > 5 and all(c not in CHARS for c in generated[-5:]):
                break

        return ''.join(generated)

# 7. Тестирование генерации
print("\n=== Генерация текста в стиле Симпсонов ===")
seeds = ["homer", "lisa", "marge", "bart", "doh", "hey", "what"]
for seed in seeds:
    gen = generate_text(seed, max_length=40, temperature=0.8)
    print(f"Seed: '{seed}' → '{gen}'")

/tmp/ipython-input-2264100089.py:8: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('simpsons_script_lines.csv')


Размер словаря: 29
Форма данных: torch.Size([132075, 50])
Epoch 1/20. Time: 58.58s, Loss: 1.6826
Epoch 2/20. Time: 58.15s, Loss: 1.5212
Epoch 3/20. Time: 57.07s, Loss: 1.4953
Epoch 4/20. Time: 58.14s, Loss: 1.4828
Epoch 5/20. Time: 58.21s, Loss: 1.4758
Epoch 6/20. Time: 58.69s, Loss: 1.4707
Epoch 7/20. Time: 59.16s, Loss: 1.4678
Epoch 8/20. Time: 57.90s, Loss: 1.4647
Epoch 9/20. Time: 56.77s, Loss: 1.4632
Epoch 10/20. Time: 57.41s, Loss: 1.4612
Epoch 11/20. Time: 58.88s, Loss: 1.4607
Epoch 12/20. Time: 56.93s, Loss: 1.4617
Epoch 13/20. Time: 57.68s, Loss: 1.4947
Epoch 14/20. Time: 57.91s, Loss: 1.4683
Epoch 15/20. Time: 58.12s, Loss: 1.4624
Epoch 16/20. Time: 57.51s, Loss: 1.4612
Epoch 17/20. Time: 57.37s, Loss: 1.4614
Epoch 18/20. Time: 56.84s, Loss: 1.4742
Epoch 19/20. Time: 58.05s, Loss: 1.4674
Epoch 20/20. Time: 58.66s, Loss: 1.4638

=== Генерация текста в стиле Симпсонов ===
Seed: 'homer' → 'homery are we get another for thous po thous '
Seed: 'lisa' → 'lisaaay mavie springfiel ar